In [2]:

import datetime
from osgeo import gdal

import os
import numpy as np
import numpy.ma as ma
import pandas as pnd
import matplotlib as ml
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
import xarray as xr
import netCDF4 as nc
import pymannkendall as mk

def datetime2timestamp(dt,returnINT=True):
    s = dt.strftime('%Y%m%d%H')
    if returnINT:
        return int(s)
    else:
        return s

C:\WPy64-3950\python-3.9.5.amd64\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [3]:
GRIBFileName = r's:\Calculations\CE2COAST\GoR\gribsurf\GoR1993010100'
depthFN = r"z:\GRUPAS\Evident\STRAUMES\GoR\KisezersGoR_.xyz"

GRIB24Dir = r'Z:\GRUPAS\Evident\STRAUMES\GoR\grib24'+'\\GoR'
GRIBsurfDir = r'Z:\GRUPAS\Evident\STRAUMES\GoR\gribsurf'+'\\GoR'

In [4]:
depth = pnd.read_csv(depthFN,sep=";",header=None, names=["lon","lat","z","i","j"],index_col=False)
lon,lat,z = np.array(depth["lon"]).reshape(203,187), np.array(depth["lat"]).reshape(203,187),np.array(depth["z"]).reshape(203,187)

lonAr,latAr = lon[0,:],lat[:,0]

depthLevels = np.array([ 1.,  4.,  8., 12., 16., 20., 24., 28., 32., 36., 40., 44., 48., 52., 56.])

In [5]:
startTime = datetime.datetime(1993,1,1)
endTime = datetime.datetime(2020,12,25)

In [6]:
dti24_pnd =  pnd.date_range(start="1993-01-01", end="2020-12-30", freq="D") 
dti24 = np.array(dti24_pnd)
pnd_dt_list = pnd.DatetimeIndex(dti24) 

In [7]:
grib24Period = 82
grib24Fields = [
 ("rsn",1,'u-component of wind [m/s]'),
 ("sst", 2,'v-component of wind [m/s]'),
 ("p82.128",3,'Deviation of sea level from mean [m]'),
 ("p91.128",4,'Ice cover (ice=1;no ice=0) [proportion]'),
 ("p92.128",5,'Ice thickness [m]'),
 ("p93.128",6,'Direction of ice drift [deg]'),
 ("p94.128",7,'Speed of ice drift [m/s]'),
 ("p80.128",8,'Water Temperature [C]'),
 ("p88.128",23,'Salinity [kg/kg]*1000'),
 ("fg10",38,'u-component of current [m/s]'),
 ("lspf",53,'v-component of current [m/s]'),
 ("magss",68,'Speed of current [m/s]')]
gribsurfPeriod = 12
gribsurfFields = [
("rsn",1,'u-component of wind [m/s]'),
("sst", 2,'v-component of wind [m/s]'),
("p82.128",3,'Deviation of sea level from mean [m]'),
("p91.128",4,'Ice cover (ice=1;no ice=0) [proportion]'),
("p92.128",5,'Ice thickness [m]'),
("p93.128",6,'Direction of ice drift [deg]'),
("p94.128",7,'Speed of ice drift [m/s]'),
("p80.128",8,'Water Temperature [C]'),
("p88.128",9,'Salinity [kg/kg]'),
("fg10",10,'u-component of current [m/s]'),
("lspf",11,'v-component of current [m/s]'),
("magss",12,'Speed of current [m/s]')]

In [8]:
from netCDF4 import Dataset
ncfile = nc.Dataset("ncfile.nc", 'w')
try:
    #Dimension:
    time = ncfile.createDimension('time', None)
    depth = ncfile.createDimension('depth', depthLevels.shape[0])
    lat = ncfile.createDimension('lat', latAr.shape[0])
    lon = ncfile.createDimension('lon', lonAr.shape[0])
    cmponet = ncfile.createDimension('com', 2)
    #Coordinates:
    times = ncfile.createVariable("time","f8",("time",))
    depths = ncfile.createVariable("depth","f4",("depth",))
    latitudes = ncfile.createVariable("lat","f4",("lat",))
    longitudes = ncfile.createVariable("lon","f4",("lon",))
    componet = ncfile.createVariable("com","i4",("com",))
    bathymetry = ncfile.createVariable("bathymetry","f4",("lat","lon"))

    depths[:] = depthLevels
    latitudes[:] = latAr
    longitudes[:] = lonAr
    componet[:] = np.arange(2)
    bathymetry[:, :] = z

    shsize3 = (1, latAr.shape[0], lonAr.shape[0])
    shsize4 = (1,1, latAr.shape[0], lonAr.shape[0])
    shsize41 = (1, latAr.shape[0], lonAr.shape[0], 2)
    shsize5 = (1,1, latAr.shape[0], lonAr.shape[0], 2)
    #
    dim4 = ("time", "depth", "lat", "lon")
    dim3 = ("time", "lat", "lon")
    dim41 = ("time", "lat", "lon", "com")
    dim5 = ("time", "depth", "lat", "lon", "com")
    #Variables:
    #8, Water Temperature [C]
    temperature = ncfile.createVariable("thetao", 'f4', dim4, zlib=True, chunksizes=shsize4, least_significant_digit=2)
    #23, Salinity [kg/kg]*1000
    salinity = ncfile.createVariable("so", 'f4', dim4, zlib=True, chunksizes=shsize4, least_significant_digit=2)
    #3, Deviation of sea level from mean [m]
    level = ncfile.createVariable("sla", 'f4', dim3, zlib=True, chunksizes=shsize3, least_significant_digit=3)
    #1, u-component of wind [m/s]. 2, v-component of wind [m/s]
    wind = ncfile.createVariable("wind", 'f4', dim41, zlib=True, chunksizes=shsize41, least_significant_digit=2)
    #4, Ice cover (ice=1;no ice=0) [proportion]
    ice_cover = ncfile.createVariable("ice_cover", 'f4', dim3, zlib=True, chunksizes=shsize3, least_significant_digit=4)
    #5, Ice thickness [m]
    ice_thinkness = ncfile.createVariable("ice_thinkness", 'f4', dim3, zlib=True, chunksizes=shsize3, least_significant_digit=3)
    #6, Direction of ice drift [deg]
    dirof_ice_drift = ncfile.createVariable("dirof_ice_drift", 'f4', dim3, zlib=True, chunksizes=shsize3, least_significant_digit=1)
    #7, Speed of ice drift [m/s]
    speedof_ice_drift = ncfile.createVariable("speedof_ice_drift", 'f4', dim3, zlib=True, chunksizes=shsize3, least_significant_digit=3)
    #38, u-component of current [m/s]. 53,v-component of current [m/s]
    current = ncfile.createVariable("current", 'f4', dim5, zlib=True, chunksizes=shsize5, least_significant_digit=3)
    #68, Speed of current [m/s]
    speed_of_current = ncfile.createVariable("speed_of_current", 'f4', dim4, zlib=True, chunksizes=shsize4, least_significant_digit=6)



    level.setncatts({"long_name" : "Sea level anomalies",
    "standard_name" : "sea_surface_height_above_sea_level",
    "units" : "m"})

    depths.setncatts({"standard_name" : "depth",
    "long_name" : "Depth of each measurement",
    "units" : "m",
    "positive" : "down",
    "axis" : "Z",
    "unit_long" : "meters",
    "reference" : "sea_level SeaDataNet L111",
    "QC_procedure" : 1,
    "QC_indicator" : 1,
    "_CoordinateAxisType" : "Height",
    "_CoordinateZisPositive" : "down"})

    latitudes.setncatts({"standard_name" : "latitude",
    "long_name" : "Latitude of each location",
    "units" : "degrees_north",
    "axis" : "Y"})

    longitudes.setncatts({"standard_name" : "longitude",
    "long_name" : "Longitude of each location",
    "units" : "degrees_east",
    "axis" : "X"})

    temperature.setncatts({"standard_name" : "sea_water_potential_temperature",
    "long_name" : "potential temperature",
    "units" : "degree_Celsius",
    "unit_long" : "degree Celsius"})

    salinity.setncatts({"standard_name" : "sea_water_salinity",
    "long_name" : "salinity",
    "units" : 1e-3,
    "unit_long" : 1e-3})

    #times[:] = dti24
    calendar = 'standard'
    units = 'days since 1970-01-01 00:00'
    times[:] = nc.date2num(dti24_pnd.to_pydatetime(), units=units, calendar=calendar)
    times.setncatts({"standard_name" : "time",
    "long_name" : "time",
    "axis" : "T", 
    "calendar": calendar, "units": units})

    w = np.empty(shape=(latAr.shape[0], lonAr.shape[0],2), dtype=np.float32)
    t = startTime
    istep = 0
    while t<=endTime:
        if istep%100 == 0:
            print(f"{istep},",end="")
        GRIB_FN = GRIB24Dir +  datetime2timestamp(t,False)
        if os.path.exists(GRIB_FN):
            #ds = xr.open_dataset(GRIB_FN, engine='cfgrib')
            dataset = gdal.Open(GRIB_FN, gdal.GA_ReadOnly)
            for ti in range(5):
                msg = dataset.GetRasterBand(3+0+ti*grib24Period)
                no_data_val = msg.GetNoDataValue()
                limenis = msg.ReadAsArray()
                maska =  (limenis<-2) | (limenis > 4) | (limenis==no_data_val)
                limenis[maska] = np.nan
                level[istep, :, :] = limenis
                #
                msg = dataset.GetRasterBand(4+0+ti*grib24Period)
                no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                ice_c = msg.ReadAsArray()#.flatten()
                maska1 =  (ice_c<0) | (ice_c > 1) | (ice_c==no_data_val)
                ice_c[maska1]= np.nan
                ice_cover[istep, :, :] = ice_c
                #
                msg = dataset.GetRasterBand(5+0+ti*grib24Period)
                no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                ice_t = msg.ReadAsArray()#.flatten()
                maska1 =  (ice_t<0) | (ice_t > 10) | (ice_t==no_data_val)
                ice_t[maska1]= np.nan
                ice_thinkness[istep, :, :] = ice_t
                #
                msg = dataset.GetRasterBand(7+0+ti*grib24Period)
                no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                soi_drift = msg.ReadAsArray()#.flatten()
                maska1 =  (soi_drift<0) | (soi_drift >10) | (soi_drift==no_data_val)
                soi_drift[maska1]= np.nan
                speedof_ice_drift[istep, :, :] = soi_drift

                msg = dataset.GetRasterBand(6+0+ti*grib24Period)
                no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                ice_d = msg.ReadAsArray()#.flatten()
                maska1 =  (ice_d<-360) | (ice_d > 360) | (ice_d==no_data_val)
                ice_d[maska1]= np.nan
                dirof_ice_drift[istep, :, :] = ice_d
                #
                msg = dataset.GetRasterBand(1+0+ti*grib24Period)
                no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                wnd = msg.ReadAsArray()#.flatten()
                maska1 =  (wnd<-50) | (wnd > 50) | (wnd==no_data_val)
                wnd[maska1]= np.nan
                wnd1 = wnd
                #
                msg = dataset.GetRasterBand(2+0+ti*grib24Period)
                no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                wnd = msg.ReadAsArray()#.flatten()
                maska1 = (wnd < -50) | (wnd > 50) | (wnd == no_data_val)
                wnd[maska1] = np.nan
                w[:,:,0]=wnd1
                w[:,:,1]=wnd
                wind[istep, :, :, :] = w
                for zi in range(15):
                    msg = dataset.GetRasterBand(8+zi+ti*grib24Period)
                    no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                    temp = msg.ReadAsArray()#.flatten()
                    temp = temp+273.15
                    maska = (temp < -5) | (temp > 30) | (temp == no_data_val)
                    temp[maska] = np.nan
                    temperature[istep, zi, :, :] = temp
                    #
                    msg = dataset.GetRasterBand(23+zi+ti*grib24Period)
                    no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                    sal = msg.ReadAsArray()#.flatten()
                    maska1 = (sal < 0) | (sal > 10) | (sal == no_data_val)
                    sal[maska1] = np.nan
                    salinity[istep, zi, :, :] = sal
                    #
                    msg = dataset.GetRasterBand(38+zi+ti*grib24Period)
                    no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                    crr = msg.ReadAsArray()#.flatten()
                    maska1 = (crr < -10) | (crr > 10) | (crr == no_data_val)
                    crr[maska1] = np.nan
                    crr1 = crr
                    #
                    msg = dataset.GetRasterBand(53+zi+ti*grib24Period)
                    no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                    crr = msg.ReadAsArray()#.flatten()
                    maska1 = (crr < -10) | (crr > 10) | (crr == no_data_val)
                    crr[maska1] = np.nan
                    w[:,:,0]=crr1
                    w[:,:,1]=crr
                    current[istep, zi, :, :, :] = w
                    #
                    msg = dataset.GetRasterBand(68+zi+ti*grib24Period)
                    no_data_val = msg.GetNoDataValue() #funkcija lasa, kā tiek atzīmēs, kas norāda uz datu neeksistēšanu
                    socrr = msg.ReadAsArray()#.flatten()
                    maska1 = (socrr < -10) | (socrr > 10) | (socrr == no_data_val)
                    socrr[maska1] = np.nan
                    speed_of_current[istep, zi, :, :] = socrr
                istep += 1
                ncfile.sync()
            del dataset
        else:
            print(GRIB_FN)
        #if istep>10: break
        #break
        t = t+datetime.timedelta(days=5)
    print(istep)
finally:
    ncfile.close()

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,5000,5100,5200,5300,5400,5500,5600,5700,5800,5900,6000,6100,6200,6300,6400,6500,6600,6700,6800,6900,7000,7100,7200,7300,7400,7500,7600,7700,7800,7900,8000,8100,8200,8300,8400,8500,8600,8700,8800,8900,9000,9100,9200,9300,9400,9500,9600,9700,9800,9900,10000,10100,10200,10225
